 # Проект.
 <b>Описание проекта</b>

Наша компания решила быть на волне и купила крупный сервис для чтения книг по подписке. Проанализируем базу данных.
В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. С помощью этих данных сформулируем ценностное предложение для нового продукта.

## Шаг. Откроем файлы с данными и изучим общую информацию.

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import text, create_engine
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})
# чтобы выполнить SQL-запрос, используем Pandas
books = '''SELECT * FROM books LIMIT 5'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(books), con = con)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [2]:
authors = '''SELECT * FROM authors LIMIT 5'''
pd.io.sql.read_sql(sql=text(authors), con = con)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [3]:
publishers = '''SELECT * FROM publishers LIMIT 5'''
pd.io.sql.read_sql(sql=text(publishers), con = con)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [4]:
ratings = '''SELECT * FROM ratings LIMIT 5'''
pd.io.sql.read_sql(sql=text(ratings), con = con)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [5]:
reviews = '''SELECT * FROM reviews LIMIT 5'''
pd.io.sql.read_sql(sql=text(reviews), con = con)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


## Шаг. Посчитаем, сколько книг вышло после 1 января 2000 года;

In [6]:
total_books = '''
    SELECT COUNT(*) AS total_books
    FROM books
    WHERE publication_date > '2000-01-01'
'''

pd.io.sql.read_sql(sql=text(total_books), con = con)

,total_books
0,819


**Вывод**

819 книг вышло после 1 января 2000 года.

## Шаг. Для каждой книги посчитаем количество обзоров и среднюю оценку;

In [7]:
reviews = '''
SELECT
    b.book_id,
    b.title,
    COUNT(r.review_id) AS review_count,
    AVG(rt.rating) AS average_rating
FROM
    books b
LEFT JOIN
    reviews r ON b.book_id = r.book_id
LEFT JOIN
    ratings rt ON b.book_id = rt.book_id
GROUP BY
    b.book_id, b.title;
'''
pd.io.sql.read_sql(sql=text(reviews), con = con)

,book_id,title,review_count,average_rating
0,652,The Body in the Library (Miss Marple #3),4,4.500000
1,273,Galápagos,4,4.500000
2,51,A Tree Grows in Brooklyn,60,4.250000
3,951,Undaunted Courage: The Pioneering First Missio...,4,4.000000
4,839,The Prophet,28,4.285714
...,...,...,...,...
995,64,Alice in Wonderland,52,4.230769
996,55,A Woman of Substance (Emma Harte Saga #1),4,5.000000
997,148,Christine,21,3.428571
998,790,The Magicians' Guild (Black Magician Trilogy #1),4,3.500000


**Вывод**

Для 1000 книг посчитали количество обзоров и среднюю оценку.

## Шаг. Определим издательство, которое выпустило наибольшее число книг толще 50 страниц — так мы исключим из анализа брошюры;

In [8]:
publisher = '''
SELECT
    p.publisher_id,
    p.publisher,
    COUNT(b.book_id) AS book_count
FROM
    publishers p
JOIN
    books b ON p.publisher_id = b.publisher_id
WHERE
    b.num_pages > 50
GROUP BY
    p.publisher_id, p.publisher
ORDER BY
    book_count DESC
LIMIT 1;
'''
pd.io.sql.read_sql(sql=text(publisher), con = con)

,publisher_id,publisher,book_count
0,212,Penguin Books,42


**Вывод**

Издательство Penguin Books наибольшее число книг толще 50 страниц. Оно выпустило 42 книги.

## Шаг. Определим автора с самой высокой средней оценкой книг — учитываем только книги с 50 и более оценками;

In [9]:
author = '''
SELECT
    a.author_id,
    a.author,
    AVG(rating.rating) AS average_rating
FROM
    authors a
JOIN
    books b ON a.author_id = b.author_id
JOIN
    (
        SELECT
            book_id,
            AVG(rating) AS rating
        FROM
            ratings
        GROUP BY
            book_id
        HAVING
            COUNT(*) >= 50
    ) rating ON b.book_id = rating.book_id
GROUP BY
    a.author_id, a.author
ORDER BY
    average_rating DESC
LIMIT 1;
'''
pd.io.sql.read_sql(sql=text(author), con = con)

,author_id,author,average_rating
0,236,J.K. Rowling/Mary GrandPré,4.283844


**Вывод**

Автор с самой высокой средней оценкой книг, у которого книги с 50 и более оценками является J.K. Rowling/Mary GrandPré и средним рейтингом 4,28.

## Шаг. Посчитаем среднее количество обзоров от пользователей, которые поставили больше 48 оценок.

In [10]:
review_count = '''
WITH i AS (
    SELECT
        COUNT(reviews.review_id) AS reviews_count
    FROM
        reviews
    WHERE
        username IN (
            SELECT
                username
            FROM
                ratings
            GROUP BY
                username
            HAVING
                COUNT(rating_id) > 48
        )
    GROUP BY
        username
)

SELECT
    ROUND(AVG(i.reviews_count)) AS avg_reviews
FROM
    i
'''
pd.io.sql.read_sql(sql=text(review_count), con = con)

,avg_reviews
0,24.0


**Вывод**

В среднем 24 обзора от пользователей, которые поставили больше 48 оценок.